In [2]:
# テキストを機械学習向けの表現に処理する前に、よくあるテキストデータの種類を把握する
# 1. カテゴリデータ 2. 意味的にはカテゴリに分類できる自由に書かれた文字列 3 構造化された文字列 4 テキストデータ

In [3]:
# サンプルデータは http://ai.stanford.edu/~amaas/data/sentiment/ から取得する

from sklearn.datasets import load_files

reviews_train = load_files('data/aclImdb/train/')
# load_files は一連の訓練テキストと訓練ラベルを返す
text_train, y_train, = reviews_train.data, reviews_train.target
print(f"type of text_train: {type(text_train)}")
print(f"length of text_train: {len(text_train)}")
print(f"text_train[1]: {text_train[1]}")

type of text_train: <class 'list'>
length of text_train: 25000
text_train[1]: b'Words can\'t describe how bad this movie is. I can\'t explain it by writing only. You have too see it for yourself to get at grip of how horrible a movie really can be. Not that I recommend you to do that. There are so many clich\xc3\xa9s, mistakes (and all other negative things you can imagine) here that will just make you cry. To start with the technical first, there are a LOT of mistakes regarding the airplane. I won\'t list them here, but just mention the coloring of the plane. They didn\'t even manage to show an airliner in the colors of a fictional airline, but instead used a 747 painted in the original Boeing livery. Very bad. The plot is stupid and has been done many times before, only much, much better. There are so many ridiculous moments here that i lost count of it really early. Also, I was on the bad guys\' side all the time in the movie, because the good guys were so stupid. "Executive Decisio

In [4]:
# 改行コードを削除する
import numpy as np

text_train = [doc.replace(b'<br />', b' ') for doc in text_train]
print(f"Samples per class (training): {np.bincount(y_train)}")

Samples per class (training): [12500 12500]


In [5]:
# テストデータもロードする
reviews_test = load_files('data/aclImdb/test/')
text_test, y_test = reviews_test.data, reviews_test.target
print(f"Number of documents in test data: {len(text_test)}")
print(f"Samples per class (test): {np.bincount(y_test)}")
text_test = [doc.replace(b'<br />', b' ') for doc in text_test]

Number of documents in test data: 25000
Samples per class (test): [12500 12500]


In [6]:
# テキストデータに対して、「肯定的」または「否定的」のラベルをつける
# BoWを使う
# コーパスに対して、BoWを適用するには次の3ステップが必要になる
# 1. トークン分割: 個々の文書（トークン）をホワイトスペースや句読点で区切る
# 2. ボキャブラリ構築: すべての単語をボキャブラリとして集め、番号をつける
# 3.エンコード: 個々の文書に対してボキャブラリの単語が現れる回数を数える

In [7]:
# トイデータセットに対してBoWを適用してみる
bards_word = ['The fool doth think he is wise,', 'but the wise man knows himself to be a fool']

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(bards_word)
print(f"Vocabulary size: {len(vect.vocabulary_)}")
print(f"Vocabulary content: {vect.vocabulary_}")

Vocabulary size: 13
Vocabulary content: {'the': 9, 'fool': 3, 'doth': 2, 'think': 10, 'he': 4, 'is': 6, 'wise': 12, 'but': 1, 'man': 8, 'knows': 7, 'himself': 5, 'to': 11, 'be': 0}


In [9]:
bag_of_words = vect.transform(bards_word)
print(f"bag_of_words: {repr(bag_of_words)}")

bag_of_words: <2x13 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>


In [10]:
print(f"Dense representation of bag_of_words: {bag_of_words.toarray()}")
# 出現回数がわかる

Dense representation of bag_of_words: [[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


In [11]:
vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)
print(f"{repr(X_train)}")
# 74849個の単語から構成されていることがわかる

<25000x74849 sparse matrix of type '<class 'numpy.int64'>'
	with 3431196 stored elements in Compressed Sparse Row format>


In [12]:
feature_names = vect.get_feature_names()
print(f"Number of features: {len(feature_names)}")
print(f"First 20 features: {feature_names[:20]}")
print(f"Features 20010 to 20030: {feature_names[20010:20030]}")
print(f"Every 2000th feature: {feature_names[::2000]}")

Number of features: 74849
First 20 features: ['00', '000', '0000000000001', '00001', '00015', '000s', '001', '003830', '006', '007', '0079', '0080', '0083', '0093638', '00am', '00pm', '00s', '01', '01pm', '02']
Features 20010 to 20030: ['dratted', 'draub', 'draught', 'draughts', 'draughtswoman', 'draw', 'drawback', 'drawbacks', 'drawer', 'drawers', 'drawing', 'drawings', 'drawl', 'drawled', 'drawling', 'drawn', 'draws', 'draza', 'dre', 'drea']
Every 2000th feature: ['00', 'aesir', 'aquarian', 'barking', 'blustering', 'bête', 'chicanery', 'condensing', 'cunning', 'detox', 'draper', 'enshrined', 'favorit', 'freezer', 'goldman', 'hasan', 'huitieme', 'intelligible', 'kantrowitz', 'lawful', 'maars', 'megalunged', 'mostey', 'norrland', 'padilla', 'pincher', 'promisingly', 'receptionist', 'rivals', 'schnaas', 'shunning', 'sparse', 'subset', 'temptations', 'treatises', 'unproven', 'walkman', 'xylophonist']


In [13]:
# 数字が多いことがわかる。数字はデータ的に不要そうなので、省いて良いかもしれないが、007は映画の意味もあるため、そう簡単に見極められない
# draも複数系の単語として意味が重複していることがあるので、別の特徴量としてカウントするのは理想的ではない

In [14]:
# 特徴量を改良する前に、クラス分類器を構築して、性能の定量的な指標を得ておこう
# 訓練ラベルはy_trainに格納されており、訓練データのBoW表現はX_trainに格納されているので、クラス分類器を訓練することができる
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import numpy as np
scores = cross_val_score(LogisticRegression(), X_train, y_train, cv=5)
print(f"Mean cross-validation accuracy: {np.mean(scores)}")

Mean cross-validation accuracy: 0.88132


In [15]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)
# 最良のパラメータを探す
print(f"Best cross-validation score: {grid.best_score_}")
print(f"Best parameters: {grid.best_params_}")

Best cross-validation score: 0.88816
Best parameters: {'C': 0.1}


In [16]:
X_test = vect.transform(text_test)
print(f"{grid.score(X_test, y_test)}")

0.87892


In [17]:
# 単語の抽出を改善できるか試す
# 5つ以上の文書に現れたものだけをトークンとする
vect = CountVectorizer(min_df=5).fit(text_train)
X_train = vect.transform(text_train)
print(f"X_train with min_df: {repr(X_train)}")

feature_names = vect.get_feature_names()
print(f"First 50 features: {feature_names[:50]}")
print(f"Features 20010 to 20030: {feature_names[20010:20030]}")
print(f"Every 700th feature: {feature_names[::700]}")

X_train with min_df: <25000x27271 sparse matrix of type '<class 'numpy.int64'>'
	with 3354014 stored elements in Compressed Sparse Row format>
First 50 features: ['00', '000', '007', '00s', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '100', '1000', '100th', '101', '102', '103', '104', '105', '107', '108', '10s', '10th', '11', '110', '112', '116', '117', '11th', '12', '120', '12th', '13', '135', '13th', '14', '140', '14th', '15', '150', '15th', '16', '160', '1600', '16mm', '16s', '16th']
Features 20010 to 20030: ['repentance', 'repercussions', 'repertoire', 'repetition', 'repetitions', 'repetitious', 'repetitive', 'rephrase', 'replace', 'replaced', 'replacement', 'replaces', 'replacing', 'replay', 'replayable', 'replayed', 'replaying', 'replays', 'replete', 'replica']
Every 700th feature: ['00', 'affections', 'appropriately', 'barbra', 'blurbs', 'butchered', 'cheese', 'commitment', 'courts', 'deconstructed', 'disgraceful', 'dvds', 'eschews', 'fell', 'freezer', 'goriest',

In [18]:
from sklearn.linear_model import LogisticRegression
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)
print(f"Best cross-validation score: {grid.best_score_}")

Best cross-validation score: 0.88812


In [19]:
# ストップワード
# 役にたたない単語を取り除くもうひとつの手段として、あまりに頻出して役に立たない単語を捨てる方法がある
# 1: 言語固有のストップワードリストを作っておく
# 2: 頻度の高い単語を捨てる
# skleanでは英語のストップワードリストをもっている
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
print(f"Number of stop words: {len(ENGLISH_STOP_WORDS)}")
print(f"Every 10th stopword: {list(ENGLISH_STOP_WORDS)[::10]}")

Number of stop words: 318
Every 10th stopword: ['behind', 'by', 'out', 'rather', 'around', 'everything', 'the', 'never', 'third', 'get', 'give', 'although', 'de', 'someone', 'full', 'him', 'amount', 'among', 'serious', 'other', 'hence', 'been', 'own', 'could', 'only', 'those', 'for', 'un', 'this', 'onto', 'beforehand', 'until']


In [20]:
# 組み込みのstopwrodリストを使う
vect = CountVectorizer(min_df=5, stop_words='english').fit(text_train)
X_train = vect.transform(text_train)
print(f"X_train with stp words: {repr(X_train)}")

X_train with stp words: <25000x26966 sparse matrix of type '<class 'numpy.int64'>'
	with 2149958 stored elements in Compressed Sparse Row format>


In [21]:
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)
# 今回のケースではストップワードによる恩恵は少ない。ただ、少しダけ処理を早くすることができる。
print(f"Best cross-validation score: {grid.best_score_}")

Best cross-validation score: 0.88296


In [22]:
# tf-idfを用いたデータのスケール変換
# 特徴量がどの程度情報を持っていそうかに応じて、特徴量をスケールを変換する手法がある
# 一般的な手法として tf-idfがある
# 特定の文書にだけ頻繁に現れる単語に大きな重みを与え、コーパスの中の多数の文書に現れる単語にはあまり重みを与えない
# つまり、特定の文書にあけ頻出し、他の文書にはあまり現れない単語は、その文書の内容をよく示しているのではないか、という発想
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(TfidfVectorizer(min_df=5, norm=None), LogisticRegression())

param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10]} 

grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(text_train, y_train)
print(f"Best cross-validation score: {grid.best_score_}")


Best cross-validation score: 0.89392


In [23]:
vectorizer = grid.best_estimator_.named_steps['tfidfvectorizer']
# 訓練データセットを変換
X_train = vectorizer.transform(text_train)
# それぞれの特徴量のデータセット中での最大値を見つける
max_value = X_train.max(axis=0).toarray().ravel()
sorted_byidf = max_value.argsort()
# 特徴量名を取得
feature_names = np.array(vectorizer.get_feature_names())

print(f"Features with lowest tfidf: {feature_names[sorted_byidf[20:]]}")
print(f"Features with highest tfidf: {feature_names[sorted_byidf[-20:]]}")

Features with lowest tfidf: ['closest' 'staring' 'combine' ... 'rob' 'timon' 'titanic']
Features with highest tfidf: ['coop' 'homer' 'dillinger' 'hackenstein' 'gadget' 'taker' 'macarthur'
 'vargas' 'jesse' 'basket' 'dominick' 'the' 'victor' 'bridget' 'victoria'
 'khouri' 'zizek' 'rob' 'timon' 'titanic']


In [24]:
sorted_by_idf = np.argsort(vectorizer.idf_)
print(f"Features with lowest idf: {feature_names[sorted_by_idf[:100]]}")

Features with lowest idf: ['the' 'and' 'of' 'to' 'this' 'is' 'it' 'in' 'that' 'but' 'for' 'with'
 'was' 'as' 'on' 'movie' 'not' 'have' 'one' 'be' 'film' 'are' 'you' 'all'
 'at' 'an' 'by' 'so' 'from' 'like' 'who' 'they' 'there' 'if' 'his' 'out'
 'just' 'about' 'he' 'or' 'has' 'what' 'some' 'good' 'can' 'more' 'when'
 'time' 'up' 'very' 'even' 'only' 'no' 'would' 'my' 'see' 'really' 'story'
 'which' 'well' 'had' 'me' 'than' 'much' 'their' 'get' 'were' 'other'
 'been' 'do' 'most' 'don' 'her' 'also' 'into' 'first' 'made' 'how' 'great'
 'because' 'will' 'people' 'make' 'way' 'could' 'we' 'bad' 'after' 'any'
 'too' 'then' 'them' 'she' 'watch' 'think' 'acting' 'movies' 'seen' 'its'
 'him']


In [25]:
import mglearn
mglearn.tools.visualize_coefficients(grid.best_estimator_.named_steps['logisticregression'].coef_, feature_names, n_top_features=40)

In [26]:
# 1単語よりも大きい意味のBoW(n-gram)
print(f"bards_words: {bards_word}")

bards_words: ['The fool doth think he is wise,', 'but the wise man knows himself to be a fool']


In [27]:
cv = CountVectorizer(ngram_range=(1, 1)).fit(bards_word)
print(f"Vocabulary size: {len(cv.vocabulary_)}")
print(f"Vocabulary: {cv.get_feature_names()}")

Vocabulary size: 13
Vocabulary: ['be', 'but', 'doth', 'fool', 'he', 'himself', 'is', 'knows', 'man', 'the', 'think', 'to', 'wise']


In [28]:
cv = CountVectorizer(ngram_range=(2, 2)).fit(bards_word)
# 対象とするトークン列の長さを長くすると特徴量の数が増大し、特定的な特徴量となる
# トークン列のアプリケーションはたいていの場合、長さを1にした方が良い
print(f"Vocabulary size: {len(cv.vocabulary_)}")
print(f"Vocabulary: {cv.get_feature_names()}")

Vocabulary size: 14
Vocabulary: ['be fool', 'but the', 'doth think', 'fool doth', 'he is', 'himself to', 'is wise', 'knows himself', 'man knows', 'the fool', 'the wise', 'think he', 'to be', 'wise man']


In [29]:
cv = CountVectorizer(ngram_range=(1, 3)).fit(bards_word)
print(f"Vocabulary size: {len(cv.vocabulary_)}")
print(f"Vocabulary: {cv.get_feature_names()}")

Vocabulary size: 39
Vocabulary: ['be', 'be fool', 'but', 'but the', 'but the wise', 'doth', 'doth think', 'doth think he', 'fool', 'fool doth', 'fool doth think', 'he', 'he is', 'he is wise', 'himself', 'himself to', 'himself to be', 'is', 'is wise', 'knows', 'knows himself', 'knows himself to', 'man', 'man knows', 'man knows himself', 'the', 'the fool', 'the fool doth', 'the wise', 'the wise man', 'think', 'think he', 'think he is', 'to', 'to be', 'to be fool', 'wise', 'wise man', 'wise man knows']


In [ ]:
# グリッドサーチを使って、最良値を探ってみる
pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
# グリッドが比較的大きい上、トリグラムが含まれているので
# このグリッドサーチの実行にはかなり時間がかかｒる
param_grid = {'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100], 'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)]}

grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(text_train, y_train)

print(f"Best cross-validation score: {grid.best_score_}")
print(f"Best parameters: {grid.best_params_}")

In [ ]:
# 個々の単語を語感を使って表現してあげる
# 同じ単語を持つすべての単語を特定する必要がある
# 単語の末尾につく特定の形を取り除くといったようばルールベースのヒューリスティックで行う場合＝＞語幹処理
# 単語の文書での役割を考慮して行う場合＝＞見出し語化
# 語幹処理で広く用いられるPorter stemmer をnltkパッケージから使う、見出し語化にはspacyパッケージを用いる
# pip install spacy && python -m spacy download en
import spacy
import nltk

# spacyの英語モデルをロード
en_nlp = spacy.load('en')
# nltkのPorter stemmerのインスタンスを作成
stemmer = nltk.stem.PorterStemmer()

# spacyによる見出し語化とntlkによる語幹処理を比較する関数を定義
def compare_normalization(doc):
    # spacyで文書をトークン分割
    doc_spacy = en_nlp(doc)
    # spacyで見つけた見出し語を表示
    print(f"Lemmatization: {[token.lemma_ for token in doc_spacy]}")
    # Porter stemmerで見つけたトークンを表示
    print(f"Stemming: {[stemmer.stem(token.norm_.lower()) for token in doc_spacy]}")

compare_normalization("Our meeting today was worse than yesterday, I'm scared of meetings")

In [ ]:
# 技術的詳細：CountVectorizerが用いている正規表現ベースの
# トークン分割器を用いて、見出し語化だけにspacyを用いるのが望ましい
# このため、en_nlp.tokenizer (spacyのトークン分割器)を正規表現ベースのトークン分割器に置き換えている
import re
# CountVectorizerで用いられているトークン分割用の正規表現
# regexp used in CountVectorizer
regexp = re.compile('(?u)\\b\\w\\w+\\b')

# spacyの言語モデルを読み込み、トークン分割器を取り出す
en_nlp = spacy.load('en')
pld_tokenizer = en_nlp.tokenizer
# トークン分割器を先ほどの正規表現で置き換える
en_nlp.tokenizer = lambda string: old_tokenizer.tokens_from_list(regexp.findall(string))

#spacyの文書処理パイプラインを持ちてカスタムトークン分割器をつくる
# (正規表現を用いたトークン分割器を組み込んである)
def custome_tokenizer(document):
    do_spacy = en_nlp(document, entity=False, parse=False)
    return [token.lemma_ for token in doc_spacy ]

# CountVectorizerをカスタムトークン分割器を使って定義する
lemma_vect = CountVectorizer(tokenizer=custom_tokenizer, min_df=5)

In [ ]:
# 見出し語化を行う
X_train_lemma = lemma_vect.fit_transform(text_train)
print(f"X_train_lemma.shape: {X_train_lemma.shape}")

# 比較のために標準のCountVectorizerでも変換
vect = CountVectorizer(min_df=5).fit(text_train)
X_train = vect.transform(text_train)
print(f"X_train.shape: {X_train.shape}")

In [ ]:
# 見出し語化の有効性を確認するために、データの1%だけを訓練データとし、残りをテストデータとして交差検証を行う
from sklearn.model_selection import StratifiedShuffleSplit

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
cv = StratifiedShuffleSplit(n_iter=5, test_size=0.99, train_size=0.01, random_state=0)

grid = GridSearchCV(LogisticRegression(), param_grid, cv=cv)
# 標準のCountVectorizerを用いてグリッドサーチを実行
grid.fit(X_train, y_train)
print(f"Best cross-validation score (standard CountVectorizer): f{grid.best_score_}")
# 見出し語化つきで、グリッドサーチを実行
grid.fit(X_train_lemma, y_train)
print(f"Best cross-validation score(lemmatization): {grid.best_score_}")

In [ ]:
# トピックモデリングと文書クラスタリング
# それぞれの文書に対して1つのトピックを割り当てるタスクをまとめて呼ぶ言葉である

In [ ]:
# LDA(Latent Dirichlet Allocation)
# 同時に現れる頻度の高い単語の集合（トピック）を探す
# 一般的な単語が解析に影響を与え過ぎないように、一般的な単語を取り除いたほうがよいとされる
vect = CountVectorizer(max_features=10000, max_df=.15)
X = vect.fit_transform(text_train)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from numpy as np
import mglearn

lda = LatentDirichletAllocation(n_topics=10, learning_method='batch', max_iter=25, ramdom_state=0)

# ここではモデルの構築と変換を一度に行う
# 変換には時間がかかるが、同時に行うことで
# 時間を節約することができる

document_topic = lda.fit_transform(X)
# それぞれの単語のそのトピックに対する重要性を格納した components_ 属性がある
print(lda.components_.shape)

# それぞれのトピックに対して、特徴量を昇順でソート
# ソートを降順にするために[:, ::-1]
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
# vectorizerから特徴量名を取得
feature_names = np.array(vect.get_feature_names())

# 最初の10トピックを表示
mglearn.tools.print_topics(topics=range=(10), feature_names=feature_names, sorting=sorting, topics_per_chunk=5, n_words=10)


In [ ]:
# より高度なテキスト処理を行うにはspacy ntlk gensim などを使うことがおすすめ